In [1]:
import os # Operating System
import numpy as np
import pandas as pd
import datetime as dt # Datetime
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge folium=0.5.0 --yes
import folium #import folium # map rendering library

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.2

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    scikit-learn-0.20.1        |   py36h22eb022_0         5.7 MB
    liblapack-3.8.0            |      11_openblas          10 KB  conda-forge
    numpy-1.18.1               |   py36h95a1406_0         5.2 MB  conda-forge
    liblapacke-3.8.0           |      11_openblas          10 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    libopenblas-0.3.6          |       h5a2b251_2         7.7 MB
    sci

In [2]:
df_ppd = pd.read_csv("http://prod2.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-2018.csv")

In [3]:
df_ppd.head(5) 

,{7011B109-CFCA-8ED6-E053-6B04A8C075C1},280000,2018-06-04 00:00,IP4 5ES,S,N,F,3,Unnamed: 8,RANDWELL CLOSE,Unnamed: 10,IPSWICH,IPSWICH.1,SUFFOLK,A,A.1
0,{7011B109-CFCB-8ED6-E053-6B04A8C075C1},280000,2018-05-29 00:00,IP1 4BS,T,N,F,261,NaN,NORWICH ROAD,NaN,IPSWICH,IPSWICH,SUFFOLK,A,A
1,{7011B109-CFCC-8ED6-E053-6B04A8C075C1},170000,2018-04-27 00:00,IP4 4BH,T,N,F,31,NaN,PARADE ROAD,NaN,IPSWICH,IPSWICH,SUFFOLK,A,A
2,{7011B109-CFCD-8ED6-E053-6B04A8C075C1},246000,2018-05-25 00:00,IP1 6NB,S,N,F,42,NaN,ELMCROFT ROAD,NaN,IPSWICH,IPSWICH,SUFFOLK,A,A
3,{7011B109-CFCE-8ED6-E053-6B04A8C075C1},180000,2018-06-08 00:00,IP3 9LZ,T,N,F,48,NaN,WYNTERTON CLOSE,NaN,IPSWICH,IPSWICH,SUFFOLK,A,A
4,{7011B109-CFCF-8ED6-E053-6B04A8C075C1},245000,2018-05-11 00:00,IP1 4BU,T,N,F,235,NaN,NORWICH ROAD,NaN,IPSWICH,IPSWICH,SUFFOLK,A,A


In [4]:
# Assign meaningful column names
df_ppd.columns = ['TUID', 'Price', 'Date_Transfer', 'Postcode', 'Prop_Type', 'Old_New', 'Duration', 'PAON', \
                  'SAON', 'Street', 'Locality', 'Town_City', 'District', 'County', 'PPD_Cat_Type', 'Record_Status']

In [5]:
# Format the date column
df_ppd['Date_Transfer'] = df_ppd['Date_Transfer'].apply(pd.to_datetime)

# Delete all obsolete transactions which were done before 2016
df_ppd.drop(df_ppd[df_ppd.Date_Transfer.dt.year < 2016].index, inplace=True)

# Sort by Date of Sale
df_ppd.sort_values(by=['Date_Transfer'],ascending=[False],inplace=True)

In [6]:
df_ppd_london = df_ppd.query("Town_City == 'LONDON'")

# Make a list of street names in LONDON
streets = df_ppd_london['Street'].unique().tolist()

In [7]:
df_grp_price = df_ppd_london.groupby(['Street'])['Price'].mean().reset_index()

# Give meaningful names to the columns
df_grp_price.columns = ['Street', 'Avg_Price']

In [8]:
#Input your Budget's Upper Limit and Lower Limit - Find the locations df_grp_price which fits your budget
df_affordable = df_grp_price.query("(Avg_Price >= 2200000) & (Avg_Price <= 2500000)")

In [9]:
# Display the dataframe
df_affordable

,Street,Avg_Price
196,ALBION SQUARE,2450000.0
390,ANHALT ROAD,2435000.0
405,ANSDELL TERRACE,2250000.0
422,APPLEGARTH ROAD,2400000.0
855,BARONSMEAD ROAD,2375000.0
...,...,...
13715,WILFRED STREET,2410538.5
13741,WILLOW BRIDGE ROAD,2425000.0
13761,WILSON STREET,2257500.0
13789,WINCHENDON ROAD,2350000.0


In [10]:
import pandas as pd
import numpy as np
import datetime as DT
import hmac
from geopy.geocoders import Nominatim
from geopy.distance import vincenty
# import k-means from clustering stage
from sklearn.cluster import KMeans

In [11]:
for index, item in df_affordable.iterrows():
    print(f"index: {index}")
    print(f"item: {item}")
    print(f"item.Street only: {item.Street}")

index: 196
item: Street       ALBION SQUARE
Avg_Price         2.45e+06
Name: 196, dtype: object
item.Street only: ALBION SQUARE
index: 390
item: Street       ANHALT ROAD
Avg_Price      2.435e+06
Name: 390, dtype: object
item.Street only: ANHALT ROAD
index: 405
item: Street       ANSDELL TERRACE
Avg_Price           2.25e+06
Name: 405, dtype: object
item.Street only: ANSDELL TERRACE
index: 422
item: Street       APPLEGARTH ROAD
Avg_Price            2.4e+06
Name: 422, dtype: object
item.Street only: APPLEGARTH ROAD
index: 855
item: Street       BARONSMEAD ROAD
Avg_Price          2.375e+06
Name: 855, dtype: object
item.Street only: BARONSMEAD ROAD
index: 981
item: Street       BEAUCLERC ROAD
Avg_Price          2.48e+06
Name: 981, dtype: object
item.Street only: BEAUCLERC ROAD
index: 1102
item: Street       BELVEDERE DRIVE
Avg_Price           2.34e+06
Name: 1102, dtype: object
item.Street only: BELVEDERE DRIVE
index: 1215
item: Street       BICKENHALL STREET
Avg_Price           2.2085e+06
N

In [12]:

address =  "PEMBROKE STUDIOS,uk"
geolocator = Nominatim()
location = geolocator.geocode(address)
# latitude = location.latitude
# longitude = location.longitude
# print('The geograpical coordinate of London City are {}, {}.'.format(latitude, longitude))
location

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  


In [15]:
apki_value = list(df_affordable['Street'].values)


In [14]:
lats= []
longs = []
street_name = []
data_street = []
for idx, val in enumerate(apki_value):
    try:
        lats.append(geolocator.geocode(str(val)+', uk').latitude)
        longs.append(geolocator.geocode(str(val)+', uk').longitude)
        street_name.append(val)
        data_street.append(((geolocator.geocode(str(val)+', uk').latitude), (geolocator.geocode(str(val)+', uk').longitude),val))
        print(((geolocator.geocode(str(val)+', uk').latitude), (geolocator.geocode(str(val)+', uk').longitude)), idx, val)
    except:
        continue


(51.5394708, -0.0718129) 0 ALBION SQUARE
(51.4803164, -0.1668011) 1 ANHALT ROAD
(51.4998899, -0.1891027) 2 ANSDELL TERRACE
(53.7486539, -0.3266704) 3 APPLEGARTH ROAD
(51.4773147, -0.239457) 4 BARONSMEAD ROAD
(51.4995771, -0.2290331) 5 BEAUCLERC ROAD
(52.4142089, 1.7244152) 6 BELVEDERE DRIVE
(51.5212014, -0.1589082) 7 BICKENHALL STREET
(51.4483941, -0.1604676) 8 BIRCHLANDS AVENUE
(51.5899607, -0.3185249) 9 BRAMPTON GROVE
(51.5601748, -0.1954305) 10 BRIARDALE GARDENS
(51.5141653, 0.1989038) 11 BROOKWAY
(51.4482603, -0.0885073) 12 BURBAGE ROAD
(52.1455294, -0.4235933) 13 BURY WALK
(51.5083499, -0.1983276) 14 CALLCOTT STREET
(51.50141, -0.1951157) 15 CAMPDEN HILL ROAD
(52.6813749, 0.9654713) 16 CAMPION ROAD
(51.4995696, -0.1842477) 17 CANNING PLACE
(51.517804, -0.6102687) 18 CARLISLE ROAD
(51.5159153, -0.3105007) 19 CARLTON GARDENS
(52.9273979, -1.137153) 20 CARLYLE COURT
(51.5411955, -0.1558168) 21 CHALCOT SQUARE
(51.475222099999996, -0.1848412142023499) 23 CHELSEA CRESCENT
(51.599677, 0.

In [16]:
# import pandas as pd
df = pd.DataFrame()
df['Street'] = list(map(lambda x: x[2],data_street))
df['latitude'] = list(map(lambda x: x[0],data_street))
df['longitude'] = list(map(lambda x: x[1],data_street))




In [26]:
merged_left = pd.merge(left=df,right=df_affordable, how='left', left_on='Street', right_on='Street')
merged_left

,Street,latitude,longitude,Avg_Price
0,ALBION SQUARE,51.539471,-0.071813,2450000.0
1,ANHALT ROAD,51.480316,-0.166801,2435000.0
2,ANSDELL TERRACE,51.499890,-0.189103,2250000.0
3,APPLEGARTH ROAD,53.748654,-0.326670,2400000.0
4,BARONSMEAD ROAD,51.477315,-0.239457,2375000.0
...,...,...,...,...
132,WHITFIELD STREET,51.522944,-0.137931,2451000.0
133,WILFRED STREET,51.498840,-0.139295,2410538.5
134,WILLOW BRIDGE ROAD,51.543109,-0.095508,2425000.0
135,WILSON STREET,53.392560,-1.472736,2257500.0


In [30]:
df1 = pd.DataFrame(data = merged_left)


In [31]:
df1

,Street,latitude,longitude,Avg_Price
0,ALBION SQUARE,51.539471,-0.071813,2450000.0
1,ANHALT ROAD,51.480316,-0.166801,2435000.0
2,ANSDELL TERRACE,51.499890,-0.189103,2250000.0
3,APPLEGARTH ROAD,53.748654,-0.326670,2400000.0
4,BARONSMEAD ROAD,51.477315,-0.239457,2375000.0
...,...,...,...,...
132,WHITFIELD STREET,51.522944,-0.137931,2451000.0
133,WILFRED STREET,51.498840,-0.139295,2410538.5
134,WILLOW BRIDGE ROAD,51.543109,-0.095508,2425000.0
135,WILSON STREET,53.392560,-1.472736,2257500.0


In [18]:
address = 'London, UK'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London City are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of London City are 51.5073219, -0.1276474.


In [32]:
# create map of London using latitude and longitude values
map_london = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, price, street in zip(df1['latitude'], df1['longitude'], df1['Avg_Price'], df1['Street']):
    label = '{}, {}'.format(street, price)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
map_london


In [33]:
CLIENT_ID = '54OO3W1MF3GAK2S2ZSUP4JTYIKA5EOWUDVONHOX2YW1MGBWK' # Foursquare ID
CLIENT_SECRET = '4PDDVPN2ACPJ2YBZJLYNBNOYMYYTMPGUIH2CKJME1YPDSFXO' # Foursquare Secret
VERSION = '20181206' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 54OO3W1MF3GAK2S2ZSUP4JTYIKA5EOWUDVONHOX2YW1MGBWK
CLIENT_SECRET:4PDDVPN2ACPJ2YBZJLYNBNOYMYYTMPGUIH2CKJME1YPDSFXO


We can now proceed to the Modeling phase. We will analyze neighborhoods to recommend real estates where home buyers can make a real estate investment. We will then recommend profitable venues according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores.



### 4. Modeling

After exploring the dataset and gaining insights into it, we are ready to use the clustering methodology to analyze real estates. We will use the k-means clustering technique as it is fast and efficient in terms of computational cost, is highly flexible to account for mutations in real estate market in London and is accurate.

In [34]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Street', 
                  'Street Latitude', 
                  'Street Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


In [36]:
# Run the above function on each location and create a new dataframe called location_venues and display it.
location_venues = getNearbyVenues(names=df1['Street'],
                                   latitudes=df1['latitude'],
                                   longitudes=df1['longitude']
                                  )

ALBION SQUARE
ANHALT ROAD
ANSDELL TERRACE
APPLEGARTH ROAD
BARONSMEAD ROAD
BEAUCLERC ROAD
BELVEDERE DRIVE
BICKENHALL STREET
BIRCHLANDS AVENUE
BRAMPTON GROVE
BRIARDALE GARDENS
BROOKWAY
BURBAGE ROAD
BURY WALK
CALLCOTT STREET
CAMPDEN HILL ROAD
CAMPION ROAD
CANNING PLACE
CARLISLE ROAD
CARLTON GARDENS
CARLYLE COURT
CHALCOT SQUARE
CHARLES LANE
CHELSEA CRESCENT
CHESTER CLOSE NORTH
CHEYNE COURT
CHEYNE ROW
CHISWICK MALL
CLARENDON STREET
CLONCURRY STREET
COLBECK MEWS
CORNWALL TERRACE MEWS
COURT LANE GARDENS
CRESCENT GROVE
DEWHURST ROAD
DORIA ROAD
DOWNSHIRE HILL
DUCHESS WALK
ECCLESTON SQUARE MEWS
EGBERT STREET
EGERTON PLACE
FLORAL STREET
FRANK DIXON WAY
FULTON MEWS
GERARD ROAD
GERRARD ROAD
GIRDLERS ROAD
GORDON PLACE
GRAFTON SQUARE
GRAHAM TERRACE
HARRIS STREET
HAVANNAH STREET
HAZLEWELL ROAD
HERONDALE AVENUE
HIGHGATE HIGH STREET
HIGHWOOD HILL
HILLGATE PLACE
HOLLYCROFT AVENUE
HONEYWELL ROAD
HORTENSIA ROAD
HOXTON SQUARE
JACKSONS LANE
JOHN STREET
KINNERTON STREET
KNOX STREET
LADBROKE GROVE
LANCASTER ME

In [37]:
location_venues

,Street,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,ALBION SQUARE,51.539471,-0.071813,Crossfit Hackney,51.537875,-0.075192,Gym
1,ALBION SQUARE,51.539471,-0.071813,Better Health Bakery,51.537376,-0.075192,Bakery
2,ALBION SQUARE,51.539471,-0.071813,Chick 'n' Sours,51.541292,-0.076309,Fried Chicken Joint
3,ALBION SQUARE,51.539471,-0.071813,Berber & Q,51.536882,-0.075810,Middle Eastern Restaurant
4,ALBION SQUARE,51.539471,-0.071813,Tin Cafe,51.540766,-0.076200,Café
...,...,...,...,...,...,...,...
5093,WILSON STREET,53.392560,-1.472736,Tesco Express,53.388799,-1.469373,Grocery Store
5094,WINGATE ROAD,51.092557,1.179455,Currys PC World,51.093848,1.179104,Electronics Store
5095,WINGATE ROAD,51.092557,1.179455,The Hungry Horse,51.089400,1.180624,Gastropub
5096,WINGATE ROAD,51.092557,1.179455,Nisa Local,51.095157,1.184555,Supermarket


In [38]:
location_venues.groupby('Street').count()


,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Street,,,,,,
ALBION SQUARE,21,21,21,21,21,21
ANHALT ROAD,16,16,16,16,16,16
ANSDELL TERRACE,50,50,50,50,50,50
APPLEGARTH ROAD,4,4,4,4,4,4
BARONSMEAD ROAD,13,13,13,13,13,13
...,...,...,...,...,...,...
WHITFIELD STREET,100,100,100,100,100,100
WILFRED STREET,84,84,84,84,84,84
WILLOW BRIDGE ROAD,25,25,25,25,25,25


In [39]:
# get the List of Unique Categories
print('There are {} uniques categories.'.format(len(location_venues['Venue Category'].unique())))


There are 314 uniques categories.


In [40]:
location_venues.shape

(5098, 7)

In [41]:
# one hot encoding
venues_onehot = pd.get_dummies(location_venues[['Venue Category']], prefix="", prefix_sep="")

# add street column back to dataframe
venues_onehot['Street'] = location_venues['Street'] 

# move street column to the first column
fixed_columns = [venues_onehot.columns[-1]] + list(venues_onehot.columns[:-1])

#fixed_columns
venues_onehot = venues_onehot[fixed_columns]

venues_onehot.head()

,Street,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,...,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Waterfront,Windmill,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [42]:
london_grouped = venues_onehot.groupby('Street').mean().reset_index()
london_grouped


,Street,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,...,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Waterfront,Windmill,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,ALBION SQUARE,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.000000,0.00,...,0.0,0.0,0.00,0.00,0.0,0.0,0.000000,0.0,0.0,0.047619
1,ANHALT ROAD,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.000000,0.00,...,0.0,0.0,0.00,0.00,0.0,0.0,0.000000,0.0,0.0,0.000000
2,ANSDELL TERRACE,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.020000,0.00,...,0.0,0.0,0.00,0.00,0.0,0.0,0.000000,0.0,0.0,0.000000
3,APPLEGARTH ROAD,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.000000,0.00,...,0.0,0.0,0.00,0.00,0.0,0.0,0.000000,0.0,0.0,0.000000
4,BARONSMEAD ROAD,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.000000,0.00,...,0.0,0.0,0.00,0.00,0.0,0.0,0.000000,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,WHITFIELD STREET,0.0,0.0,0.0,0.0,0.01,0.0,0.01,0.000000,0.00,...,0.0,0.0,0.01,0.00,0.0,0.0,0.010000,0.0,0.0,0.000000
129,WILFRED STREET,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.011905,0.00,...,0.0,0.0,0.00,0.00,0.0,0.0,0.011905,0.0,0.0,0.000000
130,WILLOW BRIDGE ROAD,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.000000,0.04,...,0.0,0.0,0.00,0.00,0.0,0.0,0.000000,0.0,0.0,0.000000
131,WILSON STREET,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.000000,0.00,...,0.0,0.0,0.00,0.00,0.0,0.0,0.000000,0.0,0.0,0.000000


In [43]:

london_grouped.shape

(133, 315)

In [44]:
# What are the top 5 venues/facilities nearby profitable real estate investments?#
num_top_venues = 5

for hood in london_grouped['Street']:
    print("----"+hood+"----")
    temp = london_grouped[london_grouped['Street'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----ALBION SQUARE----
                       venue  freq
0                     Lounge  0.10
1                 Restaurant  0.10
2                Yoga Studio  0.05
3  Middle Eastern Restaurant  0.05
4                   Beer Bar  0.05


----ANHALT ROAD----
                  venue  freq
0                   Pub  0.25
1         Grocery Store  0.12
2        Farmers Market  0.06
3  Gym / Fitness Center  0.06
4          Cocktail Bar  0.06


----ANSDELL TERRACE----
            venue  freq
0           Hotel  0.06
1      Restaurant  0.06
2             Pub  0.06
3  Clothing Store  0.06
4            Café  0.04


----APPLEGARTH ROAD----
                  venue  freq
0             Nightclub  0.25
1          Burger Joint  0.25
2                   Pub  0.25
3                Casino  0.25
4  Pakistani Restaurant  0.00


----BARONSMEAD ROAD----
                 venue  freq
0      Thai Restaurant  0.08
1  Indie Movie Theater  0.08
2    Food & Drink Shop  0.08
3        Movie Theater  0.08
4       Farmers Mar

In [45]:
# Define a function to return the most common venues/facilities nearby real estate investments#

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


In [46]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Street']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

In [47]:
# create a new dataframe
venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['Street'] = london_grouped['Street']

for ind in np.arange(london_grouped.shape[0]):
    venues_sorted.iloc[ind, 1:] = return_most_common_venues(london_grouped.iloc[ind, :], num_top_venues)


In [48]:
venues_sorted.head()


,Street,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ALBION SQUARE,Restaurant,Lounge,Yoga Studio,Bookstore,Burger Joint,Café,Scenic Lookout,Cocktail Bar,Coffee Shop,Ramen Restaurant
1,ANHALT ROAD,Pub,Grocery Store,Gym / Fitness Center,Plaza,Pizza Place,Cocktail Bar,Diner,English Restaurant,Farmers Market,French Restaurant
2,ANSDELL TERRACE,Pub,Hotel,Restaurant,Clothing Store,Bakery,Italian Restaurant,Juice Bar,French Restaurant,English Restaurant,Café
3,APPLEGARTH ROAD,Pub,Nightclub,Burger Joint,Casino,Flower Shop,Farmers Market,Fast Food Restaurant,Film Studio,Fish & Chips Shop,Fish Market
4,BARONSMEAD ROAD,Nature Preserve,Food & Drink Shop,Farmers Market,Thai Restaurant,Bookstore,Coffee Shop,Park,Restaurant,Movie Theater,Breakfast Spot


In [49]:
venues_sorted.shape


(133, 11)

In [50]:
london_grouped.shape


(133, 315)

In [54]:
london_grouped=df1


After our inspection of venues/facilities/amenities nearby the most profitable real estate investments in London, we could begin by clustering properties by venues/facilities/amenities nearby.

In [55]:
#Distribute in 5 Clusters

# set number of clusters
kclusters = 5

london_grouped_clustering = london_grouped.drop('Street', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(london_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:50]


array([2, 4, 0, 4, 1, 2, 1, 0, 0, 2, 4, 4, 4, 2, 1, 1, 2, 4, 0, 2, 3, 3,
       4, 2, 2, 0, 4, 3, 0, 4, 1, 1, 1, 3, 4, 1, 0, 2, 1, 3, 0, 0, 0, 3,
       0, 0, 4, 2, 3, 1], dtype=int32)

In [56]:
#Dataframe to include Clusters

london_grouped_clustering=df1
london_grouped_clustering.head()

,Street,latitude,longitude,Avg_Price
0,ALBION SQUARE,51.539471,-0.071813,2450000.0
1,ANHALT ROAD,51.480316,-0.166801,2435000.0
2,ANSDELL TERRACE,51.499890,-0.189103,2250000.0
3,APPLEGARTH ROAD,53.748654,-0.326670,2400000.0
4,BARONSMEAD ROAD,51.477315,-0.239457,2375000.0


In [57]:
london_grouped_clustering.shape


(137, 4)

In [58]:
df.shape


(137, 3)

In [59]:
london_grouped_clustering.dtypes


Street        object
latitude     float64
longitude    float64
Avg_Price    float64
dtype: object

In [60]:
df.dtypes


Street        object
latitude     float64
longitude    float64
dtype: object

In [61]:
# add clustering labels
london_grouped_clustering['Cluster Labels'] = kmeans.labels_

# merge london_grouped with london_data to add latitude/longitude for each neighborhood
london_grouped_clustering = london_grouped_clustering.join(venues_sorted.set_index('Street'), on='Street')

london_grouped_clustering.head(30) # check the last columns!

,Street,latitude,longitude,Avg_Price,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ALBION SQUARE,51.539471,-0.071813,2.450000e+06,2,Restaurant,Lounge,Yoga Studio,Bookstore,Burger Joint,Café,Scenic Lookout,Cocktail Bar,Coffee Shop,Ramen Restaurant
1,ANHALT ROAD,51.480316,-0.166801,2.435000e+06,4,Pub,Grocery Store,Gym / Fitness Center,Plaza,Pizza Place,Cocktail Bar,Diner,English Restaurant,Farmers Market,French Restaurant
2,ANSDELL TERRACE,51.499890,-0.189103,2.250000e+06,0,Pub,Hotel,Restaurant,Clothing Store,Bakery,Italian Restaurant,Juice Bar,French Restaurant,English Restaurant,Café
3,APPLEGARTH ROAD,53.748654,-0.326670,2.400000e+06,4,Pub,Nightclub,Burger Joint,Casino,Flower Shop,Farmers Market,Fast Food Restaurant,Film Studio,Fish & Chips Shop,Fish Market
4,BARONSMEAD ROAD,51.477315,-0.239457,2.375000e+06,1,Nature Preserve,Food & Drink Shop,Farmers Market,Thai Restaurant,Bookstore,Coffee Shop,Park,Restaurant,Movie Theater,Breakfast Spot
5,BEAUCLERC ROAD,51.499577,-0.229033,2.480000e+06,2,Pub,Hotel,Coffee Shop,Grocery Store,Thai Restaurant,Gastropub,Supermarket,Street Food Gathering,Chinese Restaurant,Falafel Restaurant
6,BELVEDERE DRIVE,52.414209,1.724415,2.340000e+06,1,Pub,Vacation Rental,Seafood Restaurant,Fish & Chips Shop,Campground,Farm,Farmers Market,Fast Food Restaurant,Film Studio,Fish Market
7,BICKENHALL STREET,51.521201,-0.158908,2.208500e+06,0,Hotel,Café,Coffee Shop,Gastropub,Restaurant,Indian Restaurant,Pizza Place,Chinese Restaurant,Sandwich Place,Greek Restaurant
8,BIRCHLANDS AVENUE,51.448394,-0.160468,2.217000e+06,0,Pub,French Restaurant,Lake,Brewery,Chinese Restaurant,Train Station,Coffee Shop,Bakery,Dessert Shop,Food & Drink Shop
9,BRAMPTON GROVE,51.589961,-0.318525,2.456875e+06,2,Home Service,Yoga Studio,Exhibit,Farm,Farmers Market,Fast Food Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market


In [63]:
# Create Map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(london_grouped_clustering['latitude'], london_grouped_clustering['longitude'], london_grouped_clustering['Street'], london_grouped_clustering['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [68]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 0, london_grouped_clustering.columns[[3] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,2250000.0,Pub,Hotel,Restaurant,Clothing Store,Bakery,Italian Restaurant,Juice Bar,French Restaurant,English Restaurant,Café
7,2208500.0,Hotel,Café,Coffee Shop,Gastropub,Restaurant,Indian Restaurant,Pizza Place,Chinese Restaurant,Sandwich Place,Greek Restaurant
8,2217000.0,Pub,French Restaurant,Lake,Brewery,Chinese Restaurant,Train Station,Coffee Shop,Bakery,Dessert Shop,Food & Drink Shop
18,2200000.0,Bakery,Grocery Store,Food Truck,Soccer Field,Farm,Farmers Market,Fast Food Restaurant,Film Studio,Fish & Chips Shop,Fish Market
25,2250000.0,Gastropub,Construction & Landscaping,Yoga Studio,Food & Drink Shop,Farmers Market,Fast Food Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market


In [69]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 1, london_grouped_clustering.columns[[3] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,2375000.0,Nature Preserve,Food & Drink Shop,Farmers Market,Thai Restaurant,Bookstore,Coffee Shop,Park,Restaurant,Movie Theater,Breakfast Spot
6,2340000.0,Pub,Vacation Rental,Seafood Restaurant,Fish & Chips Shop,Campground,Farm,Farmers Market,Fast Food Restaurant,Film Studio,Fish Market
14,2375000.0,Pub,Park,Yoga Studio,Café,Hotel,Grocery Store,Pizza Place,Indian Restaurant,Caribbean Restaurant,Sandwich Place
15,2379652.7,Café,Italian Restaurant,Clothing Store,Bakery,Hotel,Grocery Store,Burger Joint,Restaurant,English Restaurant,Modern European Restaurant
30,2367500.0,Hotel,Pub,Garden,Coffee Shop,Café,Italian Restaurant,Mediterranean Restaurant,Chinese Restaurant,Bar,Grocery Store


In [70]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 2, london_grouped_clustering.columns[[3] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,2450000.0,Restaurant,Lounge,Yoga Studio,Bookstore,Burger Joint,Café,Scenic Lookout,Cocktail Bar,Coffee Shop,Ramen Restaurant
5,2480000.0,Pub,Hotel,Coffee Shop,Grocery Store,Thai Restaurant,Gastropub,Supermarket,Street Food Gathering,Chinese Restaurant,Falafel Restaurant
9,2456875.0,Home Service,Yoga Studio,Exhibit,Farm,Farmers Market,Fast Food Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market
13,2492500.0,Supermarket,English Restaurant,Pub,Dry Cleaner,Café,Fast Food Restaurant,Gym,Park,Hardware Store,Rental Car Location
16,2461000.0,Spa,Windmill,Soccer Field,Flower Shop,Farm,Farmers Market,Fast Food Restaurant,Film Studio,Fish & Chips Shop,Fish Market


In [71]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 3, london_grouped_clustering.columns[[3] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,2.300000e+06,Pub,Restaurant,Park,Supermarket,Pharmacy,Tennis Court,Indian Restaurant,Food & Drink Shop,Event Space,Food Truck
21,2.286679e+06,Café,Italian Restaurant,Coffee Shop,Bar,Pub,French Restaurant,Park,Yoga Studio,Playground,Speakeasy
27,2.287500e+06,Gift Shop,Pub,Brewery,Reservoir,Music Venue,Art Museum,Flower Shop,Farmers Market,Fast Food Restaurant,Film Studio
33,2.298000e+06,Pub,Burger Joint,Café,Restaurant,Bar,Grocery Store,Tapas Restaurant,Pizza Place,Gym / Fitness Center,Italian Restaurant
39,2.265000e+06,Pub,Convenience Store,Yoga Studio,Flower Shop,Farm,Farmers Market,Fast Food Restaurant,Film Studio,Fish & Chips Shop,Fish Market


In [72]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 4, london_grouped_clustering.columns[[3] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,2435000.0,Pub,Grocery Store,Gym / Fitness Center,Plaza,Pizza Place,Cocktail Bar,Diner,English Restaurant,Farmers Market,French Restaurant
3,2400000.0,Pub,Nightclub,Burger Joint,Casino,Flower Shop,Farmers Market,Fast Food Restaurant,Film Studio,Fish & Chips Shop,Fish Market
10,2397132.0,Italian Restaurant,Grocery Store,Gym / Fitness Center,Coffee Shop,Forest,Food Truck,Food Stand,Food Court,Food & Drink Shop,Flower Shop
11,2400000.0,Accessories Store,Post Office,Park,Indian Restaurant,Construction & Landscaping,Business Service,Home Service,Flea Market,Farmers Market,Fast Food Restaurant
12,2445000.0,Italian Restaurant,Athletics & Sports,Pizza Place,Food & Drink Shop,Gastropub,Stadium,Greek Restaurant,Bakery,Pub,Art Gallery


# Results and Discussion section

First of all, even though the London Housing Market may be in a rut, it is still an "ever-green" for business affairs.

We may discuss our results under two main perspectives.

First, we may examine them according to neighborhoods/London areas. It is interesting to note that, although West London (Notting Hill, Kensington, Chelsea, Marylebone) and North-West London (Hampsted) might be considered highly profitable venues to purchase a real estate according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores, South-West London (Wandsworth, Balham) and North-West London (Isliington) are arising as next future elite venues with a wide range of amenities and facilities. Accordingly, one might target under-priced real estates in these areas of London in order to make a business affair.

Second, we may analyze our results according to the five clusters we have produced. Even though, all clusters could praise an optimal range of facilities and amenities, we have found two main patterns. The first pattern we are referring to, i.e. Clusters 0, 2 and 4, may target home buyers prone to live in 'green' areas with parks, waterfronts. Instead, the second pattern we are referring to, i.e. Clusters 1 and 3, may target individuals who love pubs, theatres and soccer.

# Conclusion

To sum up, according to Bloomberg News, the London Housing Market is in a rut. It is now facing a number of different headwinds, including the prospect of higher taxes and a warning from the Bank of England that U.K. home values could fall as much as 30 percent in the event of a disorderly exit from the European Union. In this scenario, it is urgent to adopt machine learning tools in order to assist homebuyers clientele in London to make wise and effective decisions. As a result, the business problem we were posing was: how could we provide support to homebuyers clientele in to purchase a suitable real estate in London in this uncertain economic and financial scenario?

To solve this business problem, we clustered London neighborhoods in order to recommend venues and the current average price of real estate where homebuyers can make a real estate investment. We recommended profitable venues according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores.

First, we gathered data on London properties and the relative price paid data were extracted from the HM Land Registry (http://landregistry.data.gov.uk/). Moreover, to explore and target recommended locations across different venues according to the presence of amenities and essential facilities, we accessed data through FourSquare API interface and arranged them as a data frame for visualization. By merging data on London properties and the relative price paid data from the HM Land Registry and data on amenities and essential facilities surrounding such properties from FourSquare API interface, we were able to recommend profitable real estate investments.

Second, The Methodology section comprised four stages: 1. Collect Inspection Data; 2. Explore and Understand Data; 3. Data preparation and preprocessing; 4. Modeling. In particular, in the modeling section, we used the k-means clustering technique as it is fast and efficient in terms of computational cost, is highly flexible to account for mutations in real estate market in London and is accurate.

Finally, we drew the conclusion that even though the London Housing Market may be in a rut, it is still an "ever-green" for business affairs. We discussed our results under two main perspectives. First, we examined them according to neighborhoods/London areas. although West London (Notting Hill, Kensington, Chelsea, Marylebone) and North-West London (Hampsted) might be considered highly profitable venues to purchase a real estate according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores, South-West London (Wandsworth, Balham) and North-West London (Isliington) are arising as next future elite venues with a wide range of amenities and facilities. Accordingly, one might target under-priced real estates in these areas of London in order to make a business affair. Second, we analyzed our results according to the five clusters we produced. While Clusters 0, 2 and 4 may target home buyers prone to live in 'green' areas with parks, waterfronts, Clusters 1 and 3 may target individuals who love pubs, theatres and soccer.